In [4]:
import os 
from datetime import datetime
import ubiops
from dotenv import load_dotenv
import requests
import json 

# Enter your API token here
load_dotenv(".env")

client = ubiops.ApiClient(ubiops.Configuration(
	host="https://api.ubiops.com/v2.1",
	api_key={"Authorization": os.environ["TOKEN_RAYCAST_USER"]})
)

api = ubiops.CoreApi(client)

ref_datetime = datetime.strptime("2024-06-20 06:30:00", "%Y-%m-%d %H:%M:%S")

In [5]:
def raycast_data_by_query(
    token: str,
    ref_datetime: datetime
):
    file_name = f"irradiance_{ref_datetime.strftime('%Y%m%dT%H%M%S')}.nc"
    receive = requests.get(
        url=f"https://api.ubiops.com/v2.1/projects/raycast-production/buckets/predictions/files/{file_name}/download", 
        headers={'Authorization': f'{token}'}
    )

    data = json.loads(receive.content)

    return data

In [7]:
data = raycast_data_by_query(token=os.environ["TOKEN_RAYCAST_USER"], ref_datetime=ref_datetime)

In [8]:
with open("data.nc", "w") as f:
    json.dump(data, f)